In [ ]:
import pandas as pd
from archiv.utils import create_bestand

In [ ]:
file = 'legacy_data/20180810 Erfassung Staatsratsprotokolle foto (1).xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
df = pd.read_excel(file).fillna('False')

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    if row['(Archiv)Signatur'] != 'False':
        signatur = "|".join(
            [
                str(row.get('(Archiv)Signatur', default="Signatur")),
                str(row.get('Karton', default="Karton")), 
            ]
        )
        if row['Teil von Bestand'] != 'False':
            item, _ = ArchResource.objects.get_or_create(
                signature=signatur
            )

            try:
                bestand = create_bestand(row['Teil von Bestand'])[-1]
                item.location = bestand
            except Exception as e:
                print(e)            
            if row['Betreff - Titel des Dokuments'] != 'False':
                item.title = row['Betreff - Titel des Dokuments']
            if row['Zusammenfassung'] != 'False':
                item.abstract = row['Zusammenfassung']
            if row['Kommentar'] != 'False':
                item.notes = row['Kommentar']
            if row['Datum orig'] != 'False':
                item.written_date = row['Datum orig']
            if row['Typ des Dokuments'] != 'False':
                type_vocab, _ = SkosConcept.objects.get_or_create(
                    pref_label="{}".format(row['Typ des Dokuments'])
                )
                type_vocab.scheme.add(res_type_scheme)
                item.res_type = type_vocab
            if row['Datum normalisiert'] != 'False':
                try:
                    item.not_before = pd.to_datetime(row['Datum normalisiert'])
                except Exception as e:
                    print(e)
            if row['Im Dokument vorkommende Orte'] != 'False':
                for x in row['Im Dokument vorkommende Orte'].split(';'):
                    try:
                        pl = Place.objects.get(name=x.strip())
                    except Exception as e:
                        pl = None
                        print(e, x.strip())
                    if pl:
                        item.mentioned_place.add(pl)
            if row['Im Dokument erwähnte Institution'] != 'False':
                for x in row['Im Dokument erwähnte Institution'].split(';'):
                    try:
                        pl = Institution.objects.get(written_name=x.strip())
                    except Exception as e:
                        pl = None
                        print(e, x.strip())
                    if pl:
                        item.mentioned_inst.add(pl)
            if row['Im Dokument erwähnte Person'] != 'False':
                for x in row['Im Dokument erwähnte Person'].split(';'):
                    try:
                        pl = Person.objects.get(written_name=x.strip())
                    except Exception as e:
                        pl = None
                        print(e, x.strip())
                    if pl:
                        item.mentioned_person.add(pl)
            if row['Erzeuger des Dokuments (Institution)'] != 'False':
                for x in row['Erzeuger des Dokuments (Institution)'].split(';'):
                    try:
                        pl = Institution.objects.get(written_name=x.strip())
                    except Exception as e:
                        pl = None
                        print(e, x.strip())
                    if pl:
                        item.creator_inst.add(pl)
            
            item.save()